Imports

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import datetime as dt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential
import seaborn as sns
import matplotlib.pyplot as plt

In [19]:
class Prediction():
    def __init__(self, crypto_curr: str, fiat: str, start_date, end_date):
        self.crypto_curr = crypto_curr
        self.fiat = fiat
        self.start_date = start_date
        self.end_date = end_date

    def check_actual_crypto_rate(self):
        df = yf.download(f'{self.crypto_curr}-{self.fiat}', start=self.start_date, end=self.end_date)
        return df

    def prepare_dataset(self, prediction_days, scaled_data):
        x_train = []
        y_train = []
        for x in range(prediction_days, len(scaled_data)):
            x_train.append(scaled_data[x-prediction_days:x, 0])
            y_train.append(scaled_data[x, 0])
            
        #Converting into numpy array
        x_train, y_train = np.array(x_train), np.array(y_train)
        x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
        return (x_train , y_train)

In [20]:
## Loading the data
predict = Prediction("ETH","USD",dt.datetime(2023,1,1), dt.datetime.now())
data = predict.check_actual_crypto_rate()

[*********************100%***********************]  1 of 1 completed


In [21]:
### Preparing the data
prediction_days = 190
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))
x_train, y_train = predict.prepare_dataset(prediction_days, scaled_data)

## Building the model

model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=10))

model.compile(optimizer='Adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=100, batch_size=30)

''' Testing the mode '''

## Loading the test data

test_start = dt.datetime(2021, 2, 1)
test_end = dt.datetime.now()

test_data = Prediction("ETH","USD",test_start, test_end).check_actual_crypto_rate()
actual_price = test_data['Close'].values

## Concatinating the close values of the train data and the test data

total_data = pd.concat((data['Close'], test_data['Close']), axis=0)

model_inputs = total_data[len(total_data) - len(test_data) - prediction_days:].values
model_inputs = model_inputs.reshape(-1, 1)
model_inputs = scaler.transform(model_inputs)

## Making prediction from the test data

x_test = []

for x in range(prediction_days, len(model_inputs)):
    x_test.append(model_inputs[x-prediction_days:x, 0])
    
## Converting into numpy array
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

## Making predictions based on the x_test
predicted_price = model.predict(x_test)

### Reversing the scaled prices
predicted_prices = scaler.inverse_transform(predicted_price)

## Predict the next day

real_data = [model_inputs[len(model_inputs) + 1 - prediction_days:len(model_inputs+1), 0]]
real_data = np.array(real_data)
real_data = np.reshape(real_data, (real_data.shape[0], real_data.shape[1], 1))

prediction = model.predict(real_data)
prediction = scaler.inverse_transform(prediction)
print(f"The next 10 days ETH Price Prediction : {prediction}")

Epoch 1/100
2/2 [==============================] - 8s 155ms/step - loss: 0.5420
Epoch 2/100
2/2 [==============================] - 0s 145ms/step - loss: 0.4684
Epoch 3/100
2/2 [==============================] - 0s 153ms/step - loss: 0.3825
Epoch 4/100
2/2 [==============================] - 0s 146ms/step - loss: 0.2655
Epoch 5/100
2/2 [==============================] - 0s 138ms/step - loss: 0.1646
Epoch 6/100
2/2 [==============================] - 0s 155ms/step - loss: 0.1238
Epoch 7/100
2/2 [==============================] - 0s 150ms/step - loss: 0.0792
Epoch 8/100
2/2 [==============================] - 0s 155ms/step - loss: 0.0559
Epoch 9/100
2/2 [==============================] - 0s 138ms/step - loss: 0.0577
Epoch 10/100
2/2 [==============================] - 0s 145ms/step - loss: 0.0587
Epoch 11/100
2/2 [==============================] - 0s 144ms/step - loss: 0.0635
Epoch 12/100
2/2 [==============================] - 0s 144ms/step - loss: 0.0625
Epoch 13/100
2/2 [===================

In [36]:
model.evaluate(prediction, model.predict(real_data))

1/1 [==============================] - 0s 48ms/step


TypeError: in user code:

    File "c:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1852, in test_function  *
        return step_function(self, iterator)
    File "c:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1836, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1824, in run_step  **
        outputs = model.test_step(data)
    File "c:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1788, in test_step
        y_pred = self(x, training=False)
    File "c:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\danie\A